In [3]:
from datetime import datetime
from IPython.display import display
# %reload_ext autotime

import pandas as pd
import numpy as np
import re
import joblib
import sys
#import pandas_profiling

import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline

In [2]:
!{sys.executable} -m pip install pandas-profiling

     |████████████████████████████████| 261 kB 27.7 MB/s eta 0:00:01
     |████████████████████████████████| 102 kB 19.4 MB/s ta 0:00:01
     |████████████████████████████████| 675 kB 76.6 MB/s eta 0:00:01
     |████████████████████████████████| 3.1 MB 77.7 MB/s eta 0:00:01
     |████████████████████████████████| 10.2 MB 78.4 MB/s eta 0:00:01
     |████████████████████████████████| 812 kB 56.8 MB/s eta 0:00:01
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27098 sha256=a692823f7b5f635521885c7a3c3d6c666e66f2b03084352553f94522429a604b
  Stored in directory: /home/ec2-user/.cache/pip/wheels/c3/fe/0b/4450b38bceb9ae43dd7d0f16e353566f30f5f4d59a58eca2ed
  Created wheel for imagehash: filename=ImageHash-4.2.1-py2.py3-none-any.whl size=295206 sha256=48dc116cec52de649fcf0ce5a98c444d7d078c852aee384278a45f35a721fd6a
  Stored in directory: /home/ec2-user/.cache/pip/wheels/5e/77/4c/7291601777365dd7db8a3b309006122d515ee5ac086f2f5866
Successfully built htmlmin imagehash
  A

In [4]:
# Directory to access the S3 bucket

raw_s3_bucket = "fwd-sg-sagemaker-raw-data"
raw_path_dir = "ga_product_carousel"

raw_path = "s3://{raw_s3_bucket}/{raw_path_dir}"

MASTER_FILEPATH = "s3://fwd-sg-sagemaker-raw-data/ga_product_carousel"

df_session = pd.read_csv("{}/session_dataset000".format(MASTER_FILEPATH))
df_session

,full_visitor_id,client_id,session_id,quotation_id,visit_number,session_date,session_date_time,channelgrouping,source,medium,...,total_pa_pageviews,applicant_drop_out_flag,entrance_page_path,exit_page_path,entrance_page_title,exit_page_title,product_type,quote_issue_date,quote_saved_quote_indicator,quote_converted_indicator
0,3.359855e+18,GA1.3.782277269.1615969915,3359855288375164539-1615990142,51VL4523248HWUX,17,2021-03-17,2021-03-17 14:09:02,(Other),Affiliate,NonAPI,...,0,0,www.fwd.com.sg/maid-insurance/,www.fwd.com.sg/maid-insurance/online-quote/tha...,Buy Maid Insurance Plan Online | FWD Singapore,Maid Insurance Online Quote | FWD Singapore,Maid,2021-03-17 22:10:19.33,0.0,1.0
1,5.705775e+18,GA1.3.1328479377.1623220975,5705775479248675567-1623818048,NaN,5,2021-06-16,2021-06-16 04:34:08,Paid Search,google,cpc,...,0,0,www.fwd.com.sg/insurance-promotions/,www.fwd.com.sg/maid-insurance/online-quote/sum...,Insurance Promotions | FWD Singapore,Maid Insurance Online Quote | FWD Singapore,NaN,NaN,NaN,NaN
2,4.036363e+18,GA1.3.939788977.1610873841,4036362922967170033-1610873622,NaN,1,2021-01-17,2021-01-17 08:53:42,Display,dfa,cpm,...,0,0,www.fwd.com.sg/life-insurance/enhanced-term-li...,www.fwd.com.sg/life-insurance/enhanced-term-li...,Buy Term Life Insurance Online | FWD Singapore,Buy Term Life Insurance Online | FWD Singapore,NaN,NaN,NaN,NaN
3,4.736000e+18,GA1.3.1102685963.1629861488,4736000150473127536-1629861487,NaN,1,2021-08-25,2021-08-25 03:18:07,Paid Search,google,cpc,...,0,0,www.fwd.com.sg/maid-insurance/,www.fwd.com.sg/maid-insurance/,Buy Maid Insurance Plan Online | FWD Singapore,Buy Maid Insurance Plan Online | FWD Singapore,NaN,NaN,NaN,NaN
4,8.000486e+18,GA1.3.1862758392.1626312364,8000486375615860396-1626312364,NaN,1,2021-07-15,2021-07-15 01:26:04,Referral,survey.euro.confirmit.com,referral,...,0,0,www.fwd.com.sg/,www.fwd.com.sg/,"Online Insurance Company, Insurance Claims | F...","Online Insurance Company, Insurance Claims | F...",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2573010,3.555738e+18,GA1.3.827884798.1613353002,3555738133878919210-1613353002,NaN,1,2021-02-15,2021-02-15 01:36:42,Direct,(direct),(none),...,0,0,www.fwd.com.sg/,www.fwd.com.sg/fire-insurance/online-quote/qui...,"Online Insurance Company, Insurance Claims | F...",HDB Fire Plan - Instant Quote Online | FWD Sin...,NaN,NaN,NaN,NaN
2573011,1.709967e+18,GA1.3.398132808.1614584564,1709967391439231732-1614599755,NaN,3,2021-03-01,2021-03-01 11:55:55,Organic Search,google,organic,...,0,0,www.fwd.com.sg/travel-insurance/,www.fwd.com.sg/travel-insurance/online-quote/q...,Buy Travel Insurance Plan Online | FWD Singapore,Travel Insurance Online Quotes | FWD Singapore,NaN,NaN,NaN,NaN
2573012,1.545789e+18,GA1.3.359907117.1614676888,1545789298727322520-1614676889,NaN,2,2021-03-02,2021-03-02 09:21:29,Display,dfa,cpm,...,0,0,www.fwd.com.sg/life-insurance/enhanced-term-li...,www.fwd.com.sg/life-insurance/enhanced-term-li...,Buy Term Life Insurance Online | FWD Singapore,Buy Term Life Insurance Online | FWD Singapore,NaN,NaN,NaN,NaN
2573013,5.773435e+18,GA1.3.1344232586.1618669040,5773434996706176496-1618719089,AFA44590064YPKC,4,2021-04-18,2021-04-18 04:11:29,(Other),Chatbot,Chatbot_Direct,...,0,0,www.fwd.com.sg/travel-insurance/online-quote/s...,www.fwd.com.sg/travel-insurance/online-quote/s...,Travel Insurance Online Quotes | FWD Singapore,Travel Insurance Online Quotes | FWD Singapore,Travel,2021-04-17 22:23:22.12,0.0,0.0


In [6]:
df_session.to_csv("session_hits.csv", index=False)

## Session Dataset Transformations 

In [4]:
df_session = pd.read_csv('Raw_Data/session_hits.csv', dayfirst = True)
df_session = df_session[df_session['session_date_time'] >= '2021-01-01']
df_session.head()

,full_visitor_id,client_id,session_id,quotation_id,visit_number,session_date,session_date_time,channelgrouping,source,medium,...,total_pa_pageviews,applicant_drop_out_flag,entrance_page_path,exit_page_path,entrance_page_title,exit_page_title,product_type,quote_issue_date,quote_saved_quote_indicator,quote_converted_indicator
0,3.359855e+18,GA1.3.782277269.1615969915,3359855288375164539-1615990142,51VL4523248HWUX,17,2021-03-17,2021-03-17 14:09:02,(Other),Affiliate,NonAPI,...,0,0,www.fwd.com.sg/maid-insurance/,www.fwd.com.sg/maid-insurance/online-quote/tha...,Buy Maid Insurance Plan Online | FWD Singapore,Maid Insurance Online Quote | FWD Singapore,Maid,2021-03-17 22:10:19.33,0.0,1.0
1,5.705775e+18,GA1.3.1328479377.1623220975,5705775479248675567-1623818048,NaN,5,2021-06-16,2021-06-16 04:34:08,Paid Search,google,cpc,...,0,0,www.fwd.com.sg/insurance-promotions/,www.fwd.com.sg/maid-insurance/online-quote/sum...,Insurance Promotions | FWD Singapore,Maid Insurance Online Quote | FWD Singapore,NaN,NaN,NaN,NaN
2,4.036363e+18,GA1.3.939788977.1610873841,4036362922967170033-1610873622,NaN,1,2021-01-17,2021-01-17 08:53:42,Display,dfa,cpm,...,0,0,www.fwd.com.sg/life-insurance/enhanced-term-li...,www.fwd.com.sg/life-insurance/enhanced-term-li...,Buy Term Life Insurance Online | FWD Singapore,Buy Term Life Insurance Online | FWD Singapore,NaN,NaN,NaN,NaN
3,4.736000e+18,GA1.3.1102685963.1629861488,4736000150473127536-1629861487,NaN,1,2021-08-25,2021-08-25 03:18:07,Paid Search,google,cpc,...,0,0,www.fwd.com.sg/maid-insurance/,www.fwd.com.sg/maid-insurance/,Buy Maid Insurance Plan Online | FWD Singapore,Buy Maid Insurance Plan Online | FWD Singapore,NaN,NaN,NaN,NaN
4,8.000486e+18,GA1.3.1862758392.1626312364,8000486375615860396-1626312364,NaN,1,2021-07-15,2021-07-15 01:26:04,Referral,survey.euro.confirmit.com,referral,...,0,0,www.fwd.com.sg/,www.fwd.com.sg/,"Online Insurance Company, Insurance Claims | F...","Online Insurance Company, Insurance Claims | F...",NaN,NaN,NaN,NaN


In [5]:
df_session['visit_number'].describe()

count    2.201337e+06
mean     9.747362e+00
std      7.274704e+01
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      3.000000e+00
max      2.754000e+03
Name: visit_number, dtype: float64

In [6]:
print(df_session['visit_number'].quantile(0.9))
print(df_session['visit_number'].quantile(0.95))

7.0
16.0


In [7]:
outlier_bound = 10 # 7 is the 95th percentile, 20 is the 99.5th percentile

df_temp = df_session['full_visitor_id'].value_counts()
df_temp

7.913219e+18    2151
6.850989e+18    2031
5.802161e+18    1078
8.080839e+18     898
7.209912e+18     824
                ... 
5.364234e+18       1
2.957171e+18       1
4.655375e+18       1
3.706035e+17       1
9.185052e+18       1
Name: full_visitor_id, Length: 926154, dtype: int64

In [8]:
df_session = df_session[df_session['session_date_time'] >= '2021-01-01']

outlier_bound = 10A
df_temp = df_session['full_visitor_id'].value_counts()
repeat_visitors = df_temp[(df_temp>1) & (df_temp < outlier_bound)].index

df_session_small = df_session[df_session['full_visitor_id'].isin(repeat_visitors)]
print('Number of repeat sessions: {}'.format(len(df_session_small)))

print('Number of return visitors: {}'.format(df_session_small['full_visitor_id'].nunique()/
                                             df_session['full_visitor_id'].nunique()))

df_session_small.to_csv('./repeat_sessions.csv',index=False)

Number of repeat sessions: 1215047
Number of return visitors: 0.4099350647948397


In [9]:
df_session_small.sort_values(['full_visitor_id','visit_number']).head()

,full_visitor_id,client_id,session_id,quotation_id,visit_number,session_date,session_date_time,channelgrouping,source,medium,...,total_pa_pageviews,applicant_drop_out_flag,entrance_page_path,exit_page_path,entrance_page_title,exit_page_title,product_type,quote_issue_date,quote_saved_quote_indicator,quote_converted_indicator
904913,7.016968e+11,GA1.3.163.1617103223,701696772471-1617103223,NaN,1,2021-03-30,2021-03-30 11:20:23,(Other),Renewal,SMS,...,0,0,www.fwd.com.sg/maid-insurance/renewal.html?id=...,www.fwd.com.sg/maid-insurance/online-quote/ren...,Maid Insurance Online Quote | FWD Singapore,Maid Insurance Online Quote | FWD Singapore,NaN,NaN,NaN,NaN
2092096,7.016968e+11,GA1.3.163.1617103223,701696772471-1617183010,NaN,2,2021-03-31,2021-03-31 09:30:10,(Other),Renewal,SMS,...,0,0,www.fwd.com.sg/maid-insurance/renewal.html?id=...,www.fwd.com.sg/portal/,Maid Insurance Online Quote | FWD Singapore,FWD Online Login Portal | FWD Singapore,NaN,NaN,NaN,NaN
895640,7.016968e+11,GA1.3.163.1617103223,701696772471-1617183311,NaN,3,2021-03-31,2021-03-31 09:35:11,Paid Search,google,cpc,...,0,0,www.fwd.com.sg/,www.fwd.com.sg/,"Online Insurance Company, Insurance Claims | F...","Online Insurance Company, Insurance Claims | F...",NaN,NaN,NaN,NaN
1716001,7.016968e+11,GA1.3.163.1617103223,701696772471-1617183736,NaN,4,2021-03-31,2021-03-31 09:42:16,(Other),Renewal,SMS,...,0,0,www.fwd.com.sg/maid-insurance/renewal.html?id=...,www.fwd.com.sg/maid-insurance/online-quote/dem...,Maid Insurance Online Quote | FWD Singapore,Maid Insurance Online Quote | FWD Singapore,NaN,NaN,NaN,NaN
1762645,7.016968e+11,GA1.3.163.1617103223,701696772471-1617183736,8O3Z1JIIZ,4,2021-03-31,2021-03-31 09:42:16,(Other),Renewal,SMS,...,0,0,www.fwd.com.sg/maid-insurance/renewal.html?id=...,www.fwd.com.sg/maid-insurance/online-quote/dem...,Maid Insurance Online Quote | FWD Singapore,Maid Insurance Online Quote | FWD Singapore,Maid,2021-03-30 17:21:03.247,0.0,1.0


In [5]:
df_session_small = pd.read_csv('repeat_sessions.csv')

## Quotation Dataset Transformations 

In [5]:
df_quotation = pd.read_csv("{}/quotation_view000".format(MASTER_FILEPATH))
df_quotation

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
df_quotation.to_csv("quotation_hits.csv", index=False)

In [11]:
df_quotation = pd.read_csv('Raw_Data/quotation_views.csv', dayfirst = True)
df_quotation

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (2,12,13,14,15,16,19,20,22) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,full_visitor_id,client_id,quotation_id,total_sessions_count,max_visit_number,first_session_date,latest_session_date,first_session_date_time,latest_session_date_time,avg_session_quality_dim,...,product_type,quote_issue_date,quote_saved_quote_indicator,quote_converted_indicator,quote_applicant_nric,first_policy_issue_date,scv_yor_at_quote_created,scv_first_product_type,scv_issued_nfc_policies,scv_existing_customer_flag
0,1.000282e+18,GA1.3.232896385.1622111081,UGXY4911826Q70Q,1,5,2021-08-20,2021-08-20,2021-08-20 14:26:37,2021-08-20 14:26:37,12,...,Cancer,2021-08-20 22:29:26.27,0.0,0.0,S7071699B,NaN,0,NaN,0,0
1,1.000897e+17,GA1.3.23303954.1606382935,4LK24272999YCEZ,4,4,2020-11-26,2020-11-27,2020-11-26 09:28:55,2020-11-27 05:37:08,9,...,Travel,2020-11-26 17:29:46.107,0.0,0.0,NaN,NaN,0,NaN,0,0
2,1.001015e+18,GA1.3.233066960.1624073760,EEDK4734606JMWQ,1,1,2021-06-19,2021-06-19,2021-06-19 03:36:00,2021-06-19 03:36:00,3,...,Motor,2021-06-19 11:36:53.91,0.0,0.0,NaN,NaN,0,NaN,0,0
3,1.001204e+18,GA1.3.233110964.1614056480,P1751VCPA,1,5,2021-03-19,2021-03-19,2021-03-19 01:39:44,2021-03-19 01:39:44,27,...,Maid,2021-03-16 11:03:00.377,0.0,1.0,S7265448Z,2019-03-13 10:40:05.823,2,Maid,2,1
4,1.001640e+18,GA1.3.233212497.1609767885,0AE54367224YFCF,1,2,2021-01-07,2021-01-07,2021-01-07 14:00:41,2021-01-07 14:00:41,7,...,MCycle,2021-01-07 22:01:41.893,1.0,0.0,NaN,NaN,0,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1686480,9.767045e+16,GA1.3.22740674.1606702519,NaN,1,1,2020-11-30,2020-11-30,2020-11-30 02:15:19,2020-11-30 02:15:19,1,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0
1686481,9.799525e+17,GA1.3.228162962.1615301241,NaN,1,1,2021-03-09,2021-03-09,2021-03-09 14:47:21,2021-03-09 14:47:21,2,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0
1686482,9.805793e+17,GA1.3.228308906.1620344255,NaN,1,1,2021-05-07,2021-05-07,2021-05-06 23:37:43,2021-05-06 23:37:43,1,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0
1686483,9.845138e+16,GA1.3.22922497.1620632974,NaN,2,2,2021-05-10,2021-05-10,2021-05-10 07:49:34,2021-05-10 07:53:44,26,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0


In [12]:
df_id = df_quotation['full_visitor_id'].value_counts()
df_id

6.850989e+18    1698
7.913219e+18    1288
5.802161e+18     722
8.815137e+18     578
8.080839e+18     556
                ... 
4.384758e+18       1
5.956666e+17       1
8.950244e+18       1
8.450026e+18       1
6.070130e+18       1
Name: full_visitor_id, Length: 1103408, dtype: int64

In [8]:
visit_id = df_session_small['full_visitor_id'].drop_duplicates()
visit_id

0          4.736000e+18
1          2.904419e+18
2          1.357610e+18
3          4.651399e+18
4          1.919897e+18
               ...     
1213729    8.778831e+18
1213919    2.201889e+18
1214098    4.149247e+18
1214594    8.766600e+18
1214967    5.087522e+18
Name: full_visitor_id, Length: 379663, dtype: float64

In [14]:
df_quotation_small = df_quotation[df_quotation['full_visitor_id'].isin(visit_id)]

print('Number of repeat sessions: {}'.format(len(df_quotation_small)))

print('Number of return visitors: {}'.format(df_quotation_small['full_visitor_id'].nunique()/
                                             df_quotation['full_visitor_id'].nunique()))

Number of repeat sessions: 722497
Number of return visitors: 0.3326539231181938


In [15]:
df_quotation_small.to_csv('./repeat_quotation.csv',index=False)

In [16]:
df_quotation_small

,full_visitor_id,client_id,quotation_id,total_sessions_count,max_visit_number,first_session_date,latest_session_date,first_session_date_time,latest_session_date_time,avg_session_quality_dim,...,product_type,quote_issue_date,quote_saved_quote_indicator,quote_converted_indicator,quote_applicant_nric,first_policy_issue_date,scv_yor_at_quote_created,scv_first_product_type,scv_issued_nfc_policies,scv_existing_customer_flag
0,1.000282e+18,GA1.3.232896385.1622111081,UGXY4911826Q70Q,1,5,2021-08-20,2021-08-20,2021-08-20 14:26:37,2021-08-20 14:26:37,12,...,Cancer,2021-08-20 22:29:26.27,0.0,0.0,S7071699B,NaN,0,NaN,0,0
2,1.001015e+18,GA1.3.233066960.1624073760,EEDK4734606JMWQ,1,1,2021-06-19,2021-06-19,2021-06-19 03:36:00,2021-06-19 03:36:00,3,...,Motor,2021-06-19 11:36:53.91,0.0,0.0,NaN,NaN,0,NaN,0,0
4,1.001640e+18,GA1.3.233212497.1609767885,0AE54367224YFCF,1,2,2021-01-07,2021-01-07,2021-01-07 14:00:41,2021-01-07 14:00:41,7,...,MCycle,2021-01-07 22:01:41.893,1.0,0.0,NaN,NaN,0,NaN,0,0
5,1.003436e+18,GA1.3.233630727.1631075762,DE7G4666473RYUN,1,2,2021-09-30,2021-09-30,2021-09-30 08:24:35,2021-09-30 08:24:35,2,...,IH,2021-05-21 18:36:29.21,0.0,0.0,NaN,NaN,0,NaN,0,0
6,1.003507e+18,GA1.3.233647182.1611304081,23BO44028797Q7L,1,1,2021-01-22,2021-01-22,2021-01-22 08:28:02,2021-01-22 08:28:02,77,...,HDBFire,2021-01-22 16:31:09.423,0.0,1.0,S7712836J,2021-01-22 16:55:03.937,0,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1686468,9.214535e+18,GA1.3.2145426093.1625889739,NaN,1,1,2021-07-10,2021-07-10,2021-07-10 04:02:19,2021-07-10 04:02:19,7,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0
1686470,9.239602e+17,GA1.3.215126256.1618034445,NaN,2,2,2021-04-10,2021-04-10,2021-04-10 06:00:45,2021-04-10 06:45:06,26,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0
1686471,9.339511e+17,GA1.3.217452426.1614700049,NaN,4,4,2021-03-02,2021-03-03,2021-03-02 15:47:28,2021-03-03 03:35:19,4,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0
1686474,9.531720e+17,GA1.3.221927659.1628425573,NaN,1,1,2021-08-08,2021-08-08,2021-08-08 12:26:13,2021-08-08 12:26:13,52,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0


## Visitor Hits Dataset Transmations

In [13]:
df_session_small = pd.read_csv('repeat_sessions.csv')

In [14]:
df_session_small

,full_visitor_id,client_id,session_id,quotation_id,visit_number,session_date,session_date_time,channelgrouping,source,medium,...,total_pa_pageviews,applicant_drop_out_flag,entrance_page_path,exit_page_path,entrance_page_title,exit_page_title,product_type,quote_issue_date,quote_saved_quote_indicator,quote_converted_indicator
0,4.736000e+18,GA1.3.1102685963.1629861488,4736000150473127536-1629861487,NaN,1,2021-08-25,2021-08-25 03:18:07,Paid Search,google,cpc,...,0,0,www.fwd.com.sg/maid-insurance/,www.fwd.com.sg/maid-insurance/,Buy Maid Insurance Plan Online | FWD Singapore,Buy Maid Insurance Plan Online | FWD Singapore,NaN,NaN,NaN,NaN
1,2.904419e+18,GA1.3.676237732.1580681758,2904418944841894430-1616247384,ATYX4529860L6V1,2,2021-03-20,2021-03-20 13:36:24,Paid Search,google,cpc,...,0,0,www.fwd.com.sg/car-insurance/,www.fwd.com.sg/car-insurance/comprehensive-mot...,Buy Car Insurance Plan Online | FWD Singapore,Car Insurance Online Quote | FWD Singapore,Motor,2021-03-20 21:37:24.057,0.0,0.0
2,1.357610e+18,GA1.3.316093131.1628488004,1357609661763731780-1629983235,7HXO4880472DB74,2,2021-08-26,2021-08-26 13:07:15,(Other),CROSSSELL,EDM,...,0,0,www.fwd.com.sg/car-insurance/comprehensive-mot...,www.fwd.com.sg/car-insurance/comprehensive-mot...,Car Insurance Online Quote | FWD Singapore,Car Insurance Online Quote | FWD Singapore,Motor,2021-08-09 13:49:08.847,0.0,0.0
3,4.651399e+18,GA1.3.1082988166.1615619134,4651398756540638270-1615860154,NaN,3,2021-03-16,2021-03-16 02:02:34,(Other),Renewal,SMS,...,0,0,www.fwd.com.sg/maid-insurance/online-quote/ren...,www.fwd.com.sg/maid-insurance/online-quote/ren...,Maid Insurance Online Quote | FWD Singapore,Maid Insurance Online Quote | FWD Singapore,NaN,NaN,NaN,NaN
4,1.919897e+18,GA1.3.447010817.1624464357,1919896841597705189-1624464357,KZPU4748991SBMU,1,2021-06-24,2021-06-23 16:05:57,Social,m.facebook.com,referral,...,0,0,www.fwd.com.sg/commercial-car-insurance/,www.fwd.com.sg/commercial-car-insurance/,Buy Commercial Car Insurance Online | FWD Sing...,Buy Commercial Car Insurance Online | FWD Sing...,Motor,2021-06-24 00:09:37.947,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1215042,4.605354e+18,GA1.3.1072267517.1614230247,4605353919692354279-1614566027,OXLU4482229NYTY,2,2021-03-01,2021-03-01 02:33:47,Organic Search,google,organic,...,0,0,www.fwd.com.sg/home-insurance/,www.fwd.com.sg/home-insurance/online-quote/dem...,Buy Home Insurance Online | FWD Singapore,Home Insurance Online Quote | FWD Singapore,Home,2021-03-01 10:45:00.91,0.0,1.0
1215043,3.555738e+18,GA1.3.827884798.1613353002,3555738133878919210-1613353002,NaN,1,2021-02-15,2021-02-15 01:36:42,Direct,(direct),(none),...,0,0,www.fwd.com.sg/,www.fwd.com.sg/fire-insurance/online-quote/qui...,"Online Insurance Company, Insurance Claims | F...",HDB Fire Plan - Instant Quote Online | FWD Sin...,NaN,NaN,NaN,NaN
1215044,1.709967e+18,GA1.3.398132808.1614584564,1709967391439231732-1614599755,NaN,3,2021-03-01,2021-03-01 11:55:55,Organic Search,google,organic,...,0,0,www.fwd.com.sg/travel-insurance/,www.fwd.com.sg/travel-insurance/online-quote/q...,Buy Travel Insurance Plan Online | FWD Singapore,Travel Insurance Online Quotes | FWD Singapore,NaN,NaN,NaN,NaN
1215045,1.545789e+18,GA1.3.359907117.1614676888,1545789298727322520-1614676889,NaN,2,2021-03-02,2021-03-02 09:21:29,Display,dfa,cpm,...,0,0,www.fwd.com.sg/life-insurance/enhanced-term-li...,www.fwd.com.sg/life-insurance/enhanced-term-li...,Buy Term Life Insurance Online | FWD Singapore,Buy Term Life Insurance Online | FWD Singapore,NaN,NaN,NaN,NaN


In [15]:
df_quotation_small = pd.read_csv('repeat_quotation.csv')
df_quotation_small

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (2,12,13,14,15,16,19,20,22) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,full_visitor_id,client_id,quotation_id,total_sessions_count,max_visit_number,first_session_date,latest_session_date,first_session_date_time,latest_session_date_time,avg_session_quality_dim,...,product_type,quote_issue_date,quote_saved_quote_indicator,quote_converted_indicator,quote_applicant_nric,first_policy_issue_date,scv_yor_at_quote_created,scv_first_product_type,scv_issued_nfc_policies,scv_existing_customer_flag
0,1.000282e+18,GA1.3.232896385.1622111081,UGXY4911826Q70Q,1,5,2021-08-20,2021-08-20,2021-08-20 14:26:37,2021-08-20 14:26:37,12,...,Cancer,2021-08-20 22:29:26.27,0.0,0.0,S7071699B,NaN,0,NaN,0,0
1,1.001015e+18,GA1.3.233066960.1624073760,EEDK4734606JMWQ,1,1,2021-06-19,2021-06-19,2021-06-19 03:36:00,2021-06-19 03:36:00,3,...,Motor,2021-06-19 11:36:53.91,0.0,0.0,NaN,NaN,0,NaN,0,0
2,1.001640e+18,GA1.3.233212497.1609767885,0AE54367224YFCF,1,2,2021-01-07,2021-01-07,2021-01-07 14:00:41,2021-01-07 14:00:41,7,...,MCycle,2021-01-07 22:01:41.893,1.0,0.0,NaN,NaN,0,NaN,0,0
3,1.003436e+18,GA1.3.233630727.1631075762,DE7G4666473RYUN,1,2,2021-09-30,2021-09-30,2021-09-30 08:24:35,2021-09-30 08:24:35,2,...,IH,2021-05-21 18:36:29.21,0.0,0.0,NaN,NaN,0,NaN,0,0
4,1.003507e+18,GA1.3.233647182.1611304081,23BO44028797Q7L,1,1,2021-01-22,2021-01-22,2021-01-22 08:28:02,2021-01-22 08:28:02,77,...,HDBFire,2021-01-22 16:31:09.423,0.0,1.0,S7712836J,2021-01-22 16:55:03.937,0,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
722492,9.214535e+18,GA1.3.2145426093.1625889739,NaN,1,1,2021-07-10,2021-07-10,2021-07-10 04:02:19,2021-07-10 04:02:19,7,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0
722493,9.239602e+17,GA1.3.215126256.1618034445,NaN,2,2,2021-04-10,2021-04-10,2021-04-10 06:00:45,2021-04-10 06:45:06,26,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0
722494,9.339511e+17,GA1.3.217452426.1614700049,NaN,4,4,2021-03-02,2021-03-03,2021-03-02 15:47:28,2021-03-03 03:35:19,4,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0
722495,9.531720e+17,GA1.3.221927659.1628425573,NaN,1,1,2021-08-08,2021-08-08,2021-08-08 12:26:13,2021-08-08 12:26:13,52,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0


In [16]:
visit_id = df_session_small['full_visitor_id'].drop_duplicates()
visit_id

0          4.736000e+18
1          2.904419e+18
2          1.357610e+18
3          4.651399e+18
4          1.919897e+18
               ...     
1213729    8.778831e+18
1213919    2.201889e+18
1214098    4.149247e+18
1214594    8.766600e+18
1214967    5.087522e+18
Name: full_visitor_id, Length: 379663, dtype: float64

### Visitor Hits Cars

In [7]:
df_car_hits = pd.read_csv("{}/cars_visitor_hits000".format(MASTER_FILEPATH))
df_car_hits

,date,fullvisitorid,clientid,visitnumber,uniquevisitid,visitstarttime,visitdate,channelgrouping,source,medium,...,minute,isentrance,isexit,pagepath,pagepathlevel1,pagepathlevel2,pagepathlevel3,pagepathlevel4,pagetitle,type
0,20210623,7909671894890259068,GA1.3.1841613998.1624449660,1,7909671894890259068-1624449659,2021-06-23 12:00:59,2021-06-23,Direct,(direct),(none),...,15,NaN,NaN,www.fwd.com.sg/motorcycle-insurance/online-quo...,www.fwd.com.sg/,/motorcycle-insurance/,/online-quote/,/thankYou?from=notNow,Get an Online Motorcycle Insurance Quote | FWD...,PAGE
1,20210623,7909671894890259068,GA1.3.1841613998.1624449660,1,7909671894890259068-1624449659,2021-06-23 12:00:59,2021-06-23,Direct,(direct),(none),...,14,NaN,NaN,www.fwd.com.sg/motorcycle-insurance/online-quo...,www.fwd.com.sg/,/motorcycle-insurance/,/online-quote/,/paymentRedirect?policy=375434483,Get an Online Motorcycle Insurance Quote | FWD...,PAGE
2,20210623,7909671894890259068,GA1.3.1841613998.1624449660,1,7909671894890259068-1624449659,2021-06-23 12:00:59,2021-06-23,Direct,(direct),(none),...,12,NaN,NaN,www.fwd.com.sg/motorcycle-insurance/online-quo...,www.fwd.com.sg/,/motorcycle-insurance/,/online-quote/,/confirmation,Get an Online Motorcycle Insurance Quote | FWD...,PAGE
3,20210623,7909671894890259068,GA1.3.1841613998.1624449660,1,7909671894890259068-1624449659,2021-06-23 12:00:59,2021-06-23,Direct,(direct),(none),...,4,NaN,NaN,www.fwd.com.sg/motorcycle-insurance/online-quo...,www.fwd.com.sg/,/motorcycle-insurance/,/online-quote/,/application,Get an Online Motorcycle Insurance Quote | FWD...,PAGE
4,20210623,7909671894890259068,GA1.3.1841613998.1624449660,1,7909671894890259068-1624449659,2021-06-23 12:00:59,2021-06-23,Direct,(direct),(none),...,4,NaN,NaN,www.fwd.com.sg/motorcycle-insurance/online-quo...,www.fwd.com.sg/,/motorcycle-insurance/,/online-quote/,/enhancements,Get an Online Motorcycle Insurance Quote | FWD...,PAGE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2544734,20210428,296045916874956444,GA1.3.68928561.1619615388,1,296045916874956444-1619615378,2021-04-28 13:09:38,2021-04-28,Organic Search,google,organic,...,12,NaN,NaN,www.fwd.com.sg/car-insurance/comprehensive-mot...,www.fwd.com.sg/,/car-insurance/,/comprehensive-motor-insurance/,/online-quote/quickQuestions,Car Insurance Online Quote | FWD Singapore,PAGE
2544735,20210428,296045916874956444,GA1.3.68928561.1619615388,1,296045916874956444-1619615378,2021-04-28 13:09:38,2021-04-28,Organic Search,google,organic,...,12,NaN,NaN,www.fwd.com.sg/car-insurance/comprehensive-mot...,www.fwd.com.sg/,/car-insurance/,/comprehensive-motor-insurance/,/online-quote/,Car Insurance Online Quote | FWD Singapore,PAGE
2544736,20210428,296045916874956444,GA1.3.68928561.1619615388,1,296045916874956444-1619615378,2021-04-28 13:09:38,2021-04-28,Organic Search,google,organic,...,12,NaN,NaN,www.fwd.com.sg/car-insurance/comprehensive-mot...,www.fwd.com.sg/,/car-insurance/,/comprehensive-motor-insurance/,/online-quote/,Car Insurance Online Quote | FWD Singapore,PAGE
2544737,20210428,296045916874956444,GA1.3.68928561.1619615388,1,296045916874956444-1619615378,2021-04-28 13:09:38,2021-04-28,Organic Search,google,organic,...,10,NaN,NaN,www.fwd.com.sg/car-insurance/,www.fwd.com.sg/,/car-insurance/,/,NaN,Buy Car Insurance Plan Online | FWD Singapore,PAGE


In [ ]:
df_car_hits.to_csv("car_visitor_hits.csv", index=False)

In [21]:
df_car_hits['fullvisitorid'][0]

7909671894890259068

In [9]:
visit = visit_id.astype(int)
visit

0          4736000150473127936
1          2904418944841894400
2          1357609661763731712
3          4651398756540638208
4          1919896841597704704
                  ...         
1213729    8778830717046243328
1213919    2201888571581249280
1214098    4149246564871164928
1214594    8766599625136082944
1214967    5087521864778589184
Name: full_visitor_id, Length: 379663, dtype: int64

In [10]:
df_car_small = df_car_hits[df_car_hits['fullvisitorid'].isin(visit)]

print('Number of repeat sessions: {}'.format(len(df_car_small)))

print('Number of return visitors: {}'.format(df_car_small['fullvisitorid'].nunique()/
                                             df_car_hits['fullvisitorid'].nunique()))

Number of repeat sessions: 6941
Number of return visitors: 0.0024286813277638966


In [11]:
df_car_small.to_csv("car_visitor_small.csv", index=False)

### Visitor Hits Health

In [12]:
df_health_hits = pd.read_csv("{}/health_visitor_hits000".format(MASTER_FILEPATH))
df_health_hits

,date,fullvisitorid,clientid,visitnumber,uniquevisitid,visitstarttime,visitdate,channelgrouping,source,medium,...,minute,isentrance,isexit,pagepath,pagepathlevel1,pagepathlevel2,pagepathlevel3,pagepathlevel4,pagetitle,type
0,20210324,2.563789e+17,GA1.3.59692854.1616577868,1,256378857351480652-1616577867,2021-03-24 09:24:27,2021-03-24,Direct,(direct),(none),...,24,t,t,www.fwd.com.sg/life-insurance/enhanced-term-li...,www.fwd.com.sg/,/life-insurance/,/enhanced-term-life-insurance/,/,Buy Term Life Insurance Online | FWD Singapore,PAGE
1,20210324,2.454469e+18,GA1.3.571475539.1616458493,2,2454468752085431037-1616516935,2021-03-23 16:28:55,2021-03-24,Direct,(direct),(none),...,28,t,t,www.fwd.com.sg/big-3-critical-illness/,www.fwd.com.sg/,/big-3-critical-illness/,/,NaN,Buy Critical Illness Insurance Plans Online | ...,PAGE
2,20210324,3.385508e+18,GA1.3.788250014.1616564907,1,3385508032818107051-1616564907,2021-03-24 05:48:27,2021-03-24,Direct,(direct),(none),...,48,t,t,www.fwd.com.sg/life-insurance/enhanced-term-li...,www.fwd.com.sg/,/life-insurance/,/enhanced-term-life-insurance/,/,Buy Term Life Insurance Online | FWD Singapore,PAGE
3,20210324,4.195177e+18,GA1.3.976765709.1616552698,2,4195176777625805562-1616557323,2021-03-24 03:42:03,2021-03-24,Direct,(direct),(none),...,42,t,t,www.fwd.com.sg/big-3-critical-illness/,www.fwd.com.sg/,/big-3-critical-illness/,/,NaN,Buy Critical Illness Insurance Plans Online | ...,PAGE
4,20210324,5.396944e+18,GA1.3.1256573861.1616575015,1,5396943639620024871-1616575015,2021-03-24 08:36:55,2021-03-24,Direct,(direct),(none),...,36,t,t,www.fwd.com.sg/covid-19-insurance/online-quote...,www.fwd.com.sg/,/covid-19-insurance/,/online-quote/,/underwriting,Get a Covid 19 Online Quote | FWD Singapore,PAGE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1717737,20210628,2.019980e+18,GA1.3.470313307.1617695896,9,2019980274056303768-1624812865,2021-06-27 16:54:25,2021-06-28,Paid Search,google,cpc,...,54,t,NaN,www.fwd.com.sg/life-insurance/enhanced-term-li...,www.fwd.com.sg/,/life-insurance/,/enhanced-term-life-insurance/,/,Buy Term Life Insurance Online | FWD Singapore,PAGE
1717738,20210628,4.936654e+18,GA1.3.1149404215.1624850962,1,4936653514934403602-1624850962,2021-06-28 03:29:22,2021-06-28,Paid Search,google,cpc,...,30,NaN,t,www.fwd.com.sg/big-3-critical-illness/online-q...,www.fwd.com.sg/,/big-3-critical-illness/,/online-quote/,/quickQuestions,Get an Online Big 3 Critical Illness Quote | F...,PAGE
1717739,20210628,4.936654e+18,GA1.3.1149404215.1624850962,1,4936653514934403602-1624850962,2021-06-28 03:29:22,2021-06-28,Paid Search,google,cpc,...,29,t,NaN,www.fwd.com.sg/big-3-critical-illness/,www.fwd.com.sg/,/big-3-critical-illness/,/,NaN,Buy Critical Illness Insurance Plans Online | ...,PAGE
1717740,20210628,2.124225e+17,GA1.3.49458460.1624888896,1,212422469835413056-1624888904,2021-06-28 14:01:44,2021-06-28,Display,dfa,cpm,...,1,t,t,www.fwd.com.sg/life-insurance/enhanced-term-li...,www.fwd.com.sg/,/life-insurance/,/enhanced-term-life-insurance/,/family,Buy Term Life Insurance Online | FWD Singapore,PAGE


In [10]:
df_health_hits.to_csv('health_visitor_hits.csv')

In [13]:
df_health_small = df_health_hits[df_health_hits['fullvisitorid'].isin(visit)]

print('Number of repeat sessions: {}'.format(len(df_car_small)))

print('Number of return visitors: {}'.format(df_health_small['fullvisitorid'].nunique()/
                                             df_health_hits['fullvisitorid'].nunique()))

Number of repeat sessions: 6941
Number of return visitors: 0.18410414803977349


In [14]:
df_health_small.to_csv("car_visitor_small.csv", index=False)

### Visitor Hits Home

In [15]:
df_home_hits = pd.read_csv("{}/home_visitor_hits000".format(MASTER_FILEPATH))
df_home_hits

,date,fullvisitorid,clientid,visitnumber,uniquevisitid,visitstarttime,visitdate,channelgrouping,source,medium,...,minute,isentrance,isexit,pagepath,pagepathlevel1,pagepathlevel2,pagepathlevel3,pagepathlevel4,pagetitle,type
0,20210623,604727131998694971,GA1.3.140799007.1624419899,1,604727131998694971-1624419899,2021-06-23 03:44:59,2021-06-23,Direct,(direct),(none),...,15,NaN,t,www.fwd.com.sg/fire-insurance/online-quote/tha...,www.fwd.com.sg/,/fire-insurance/,/online-quote/,/thankyou,HDB Fire Plan - Instant Quote Online | FWD Sin...,PAGE
1,20210623,604727131998694971,GA1.3.140799007.1624419899,1,604727131998694971-1624419899,2021-06-23 03:44:59,2021-06-23,Direct,(direct),(none),...,15,NaN,NaN,www.fwd.com.sg/fire-insurance/online-quote/pro...,www.fwd.com.sg/,/fire-insurance/,/online-quote/,/processing/375342214/D0U44746667QABK,HDB Fire Plan - Instant Quote Online | FWD Sin...,PAGE
2,20210623,604727131998694971,GA1.3.140799007.1624419899,1,604727131998694971-1624419899,2021-06-23 03:44:59,2021-06-23,Direct,(direct),(none),...,3,NaN,NaN,www.fwd.com.sg/fire-insurance/online-quote/sum...,www.fwd.com.sg/,/fire-insurance/,/online-quote/,/summary,HDB Fire Plan - Instant Quote Online | FWD Sin...,PAGE
3,20210623,604727131998694971,GA1.3.140799007.1624419899,1,604727131998694971-1624419899,2021-06-23 03:44:59,2021-06-23,Direct,(direct),(none),...,57,NaN,NaN,www.fwd.com.sg/fire-insurance/online-quote/app...,www.fwd.com.sg/,/fire-insurance/,/online-quote/,/application,HDB Fire Plan - Instant Quote Online | FWD Sin...,PAGE
4,20210623,604727131998694971,GA1.3.140799007.1624419899,1,604727131998694971-1624419899,2021-06-23 03:44:59,2021-06-23,Direct,(direct),(none),...,57,NaN,NaN,www.fwd.com.sg/fire-insurance/online-quote/NoA...,www.fwd.com.sg/,/fire-insurance/,/online-quote/,/NoAdditions,HDB Fire Plan - Instant Quote Online | FWD Sin...,PAGE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2986170,20210104,4345427189038227533,GA1.3.1011748609.1609736269,1,4345427189038227533-1609736267,2021-01-04 04:57:47,2021-01-04,Direct,(direct),(none),...,8,NaN,NaN,www.fwd.com.sg/fire-insurance/online-quote/qui...,www.fwd.com.sg/,/fire-insurance/,/online-quote/,/quickQuestions,HDB Fire Plan - Instant Quote Online | FWD Sin...,PAGE
2986171,20210104,4345427189038227533,GA1.3.1011748609.1609736269,1,4345427189038227533-1609736267,2021-01-04 04:57:47,2021-01-04,Direct,(direct),(none),...,8,NaN,NaN,www.fwd.com.sg/fire-insurance/online-quote/,www.fwd.com.sg/,/fire-insurance/,/online-quote/,/,HDB Fire Plan - Instant Quote Online | FWD Sin...,PAGE
2986172,20210104,4345427189038227533,GA1.3.1011748609.1609736269,1,4345427189038227533-1609736267,2021-01-04 04:57:47,2021-01-04,Direct,(direct),(none),...,8,NaN,NaN,www.fwd.com.sg/fire-insurance/online-quote/,www.fwd.com.sg/,/fire-insurance/,/online-quote/,/,HDB Fire Plan - Instant Quote Online | FWD Sin...,PAGE
2986173,20210104,4345427189038227533,GA1.3.1011748609.1609736269,1,4345427189038227533-1609736267,2021-01-04 04:57:47,2021-01-04,Direct,(direct),(none),...,57,t,NaN,www.fwd.com.sg/fire-insurance/online-quote/qui...,www.fwd.com.sg/,/fire-insurance/,/online-quote/,/quickQuestions,HDB Fire Plan - Instant Quote Online | FWD Sin...,PAGE


In [12]:
df_home_hits.to_csv('home_visitor_hits.csv')

In [16]:
df_home_small = df_home_hits[df_home_hits['fullvisitorid'].isin(visit)]

print('Number of repeat sessions: {}'.format(len(df_home_small)))

print('Number of return visitors: {}'.format(df_home_small['fullvisitorid'].nunique()/
                                             df_home_hits['fullvisitorid'].nunique()))

Number of repeat sessions: 8858
Number of return visitors: 0.0023802603106934084


In [ ]:
df_home_hits.to_csv('home_visitor_hits.csv')

In [36]:
df_home_hits

,date,fullvisitorid,clientid,visitnumber,uniquevisitid,visitstarttime,visitdate,channelgrouping,source,medium,...,minute,isentrance,isexit,pagepath,pagepathlevel1,pagepathlevel2,pagepathlevel3,pagepathlevel4,pagetitle,type
0,20210623,604727131998694971,GA1.3.140799007.1624419899,1,604727131998694971-1624419899,2021-06-23 03:44:59,2021-06-23,Direct,(direct),(none),...,15,NaN,t,www.fwd.com.sg/fire-insurance/online-quote/tha...,www.fwd.com.sg/,/fire-insurance/,/online-quote/,/thankyou,HDB Fire Plan - Instant Quote Online | FWD Sin...,PAGE
1,20210623,604727131998694971,GA1.3.140799007.1624419899,1,604727131998694971-1624419899,2021-06-23 03:44:59,2021-06-23,Direct,(direct),(none),...,15,NaN,NaN,www.fwd.com.sg/fire-insurance/online-quote/pro...,www.fwd.com.sg/,/fire-insurance/,/online-quote/,/processing/375342214/D0U44746667QABK,HDB Fire Plan - Instant Quote Online | FWD Sin...,PAGE
2,20210623,604727131998694971,GA1.3.140799007.1624419899,1,604727131998694971-1624419899,2021-06-23 03:44:59,2021-06-23,Direct,(direct),(none),...,3,NaN,NaN,www.fwd.com.sg/fire-insurance/online-quote/sum...,www.fwd.com.sg/,/fire-insurance/,/online-quote/,/summary,HDB Fire Plan - Instant Quote Online | FWD Sin...,PAGE
3,20210623,604727131998694971,GA1.3.140799007.1624419899,1,604727131998694971-1624419899,2021-06-23 03:44:59,2021-06-23,Direct,(direct),(none),...,57,NaN,NaN,www.fwd.com.sg/fire-insurance/online-quote/app...,www.fwd.com.sg/,/fire-insurance/,/online-quote/,/application,HDB Fire Plan - Instant Quote Online | FWD Sin...,PAGE
4,20210623,604727131998694971,GA1.3.140799007.1624419899,1,604727131998694971-1624419899,2021-06-23 03:44:59,2021-06-23,Direct,(direct),(none),...,57,NaN,NaN,www.fwd.com.sg/fire-insurance/online-quote/NoA...,www.fwd.com.sg/,/fire-insurance/,/online-quote/,/NoAdditions,HDB Fire Plan - Instant Quote Online | FWD Sin...,PAGE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2986170,20210104,4345427189038227533,GA1.3.1011748609.1609736269,1,4345427189038227533-1609736267,2021-01-04 04:57:47,2021-01-04,Direct,(direct),(none),...,8,NaN,NaN,www.fwd.com.sg/fire-insurance/online-quote/qui...,www.fwd.com.sg/,/fire-insurance/,/online-quote/,/quickQuestions,HDB Fire Plan - Instant Quote Online | FWD Sin...,PAGE
2986171,20210104,4345427189038227533,GA1.3.1011748609.1609736269,1,4345427189038227533-1609736267,2021-01-04 04:57:47,2021-01-04,Direct,(direct),(none),...,8,NaN,NaN,www.fwd.com.sg/fire-insurance/online-quote/,www.fwd.com.sg/,/fire-insurance/,/online-quote/,/,HDB Fire Plan - Instant Quote Online | FWD Sin...,PAGE
2986172,20210104,4345427189038227533,GA1.3.1011748609.1609736269,1,4345427189038227533-1609736267,2021-01-04 04:57:47,2021-01-04,Direct,(direct),(none),...,8,NaN,NaN,www.fwd.com.sg/fire-insurance/online-quote/,www.fwd.com.sg/,/fire-insurance/,/online-quote/,/,HDB Fire Plan - Instant Quote Online | FWD Sin...,PAGE
2986173,20210104,4345427189038227533,GA1.3.1011748609.1609736269,1,4345427189038227533-1609736267,2021-01-04 04:57:47,2021-01-04,Direct,(direct),(none),...,57,t,NaN,www.fwd.com.sg/fire-insurance/online-quote/qui...,www.fwd.com.sg/,/fire-insurance/,/online-quote/,/quickQuestions,HDB Fire Plan - Instant Quote Online | FWD Sin...,PAGE


In [40]:
df_home_small.sort_values(['fullvisitorid','visitnumber'])

,date,fullvisitorid,clientid,visitnumber,uniquevisitid,visitstarttime,visitdate,channelgrouping,source,medium,...,minute,isentrance,isexit,pagepath,pagepathlevel1,pagepathlevel2,pagepathlevel3,pagepathlevel4,pagetitle,type
122663,20210330,701696772471,GA1.3.163.1617103223,1,701696772471-1617103223,2021-03-30 11:20:23,2021-03-30,(Other),Renewal,SMS,...,20,NaN,t,www.fwd.com.sg/maid-insurance/online-quote/ren...,www.fwd.com.sg/,/maid-insurance/,/online-quote/,/renewPolicy,Maid Insurance Online Quote | FWD Singapore,PAGE
122664,20210330,701696772471,GA1.3.163.1617103223,1,701696772471-1617103223,2021-03-30 11:20:23,2021-03-30,(Other),Renewal,SMS,...,20,NaN,NaN,www.fwd.com.sg/maid-insurance/renewal.html?id=...,www.fwd.com.sg/,/maid-insurance/,/renewal.html?id=560e29e5-83a2-4aa3-b7f5-a22ca...,NaN,Maid Insurance Online Quote | FWD Singapore,PAGE
122665,20210330,701696772471,GA1.3.163.1617103223,1,701696772471-1617103223,2021-03-30 11:20:23,2021-03-30,(Other),Renewal,SMS,...,20,t,NaN,www.fwd.com.sg/maid-insurance/renewal.html?id=...,www.fwd.com.sg/,/maid-insurance/,/renewal.html?id=560e29e5-83a2-4aa3-b7f5-a22ca...,NaN,Maid Insurance Online Quote | FWD Singapore,PAGE
1655997,20210331,701696772471,GA1.3.163.1617103223,2,701696772471-1617183010,2021-03-31 09:30:10,2021-03-31,(Other),Renewal,SMS,...,30,NaN,NaN,www.fwd.com.sg/maid-insurance/online-quote/ren...,www.fwd.com.sg/,/maid-insurance/,/online-quote/,/renewPolicy,Maid Insurance Online Quote | FWD Singapore,PAGE
1655998,20210331,701696772471,GA1.3.163.1617103223,2,701696772471-1617183010,2021-03-31 09:30:10,2021-03-31,(Other),Renewal,SMS,...,30,NaN,NaN,www.fwd.com.sg/maid-insurance/renewal.html?id=...,www.fwd.com.sg/,/maid-insurance/,/renewal.html?id=560e29e5-83a2-4aa3-b7f5-a22ca...,NaN,Maid Insurance Online Quote | FWD Singapore,PAGE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1998439,20211007,9135849319484439552,GA1.3.2127105677.1633500160,3,9135849319484439552-1633576696,2021-10-07 03:18:16,2021-10-07,Paid Search,google,cpc,...,18,NaN,NaN,www.fwd.com.sg/home-insurance/online-quote/own...,www.fwd.com.sg/,/home-insurance/,/online-quote/,/ownerType#owner,Home Insurance Online Quote | FWD Singapore,PAGE
1998440,20211007,9135849319484439552,GA1.3.2127105677.1633500160,3,9135849319484439552-1633576696,2021-10-07 03:18:16,2021-10-07,Paid Search,google,cpc,...,18,NaN,NaN,www.fwd.com.sg/home-insurance/online-quote/own...,www.fwd.com.sg/,/home-insurance/,/online-quote/,/ownerType#owner,Home Insurance Online Quote | FWD Singapore,PAGE
1998441,20211007,9135849319484439552,GA1.3.2127105677.1633500160,3,9135849319484439552-1633576696,2021-10-07 03:18:16,2021-10-07,Paid Search,google,cpc,...,18,NaN,NaN,www.fwd.com.sg/home-insurance/online-quote#owner,www.fwd.com.sg/,/home-insurance/,/online-quote#owner,NaN,Home Insurance Online Quote | FWD Singapore,PAGE
1998442,20211007,9135849319484439552,GA1.3.2127105677.1633500160,3,9135849319484439552-1633576696,2021-10-07 03:18:16,2021-10-07,Paid Search,google,cpc,...,18,NaN,NaN,www.fwd.com.sg/home-insurance/,www.fwd.com.sg/,/home-insurance/,/,NaN,Buy Home Insurance Online | FWD Singapore,PAGE
